In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

data_path = Path("data")

# Read the 'Monthly' sheet from the median_house_price file
prices = pd.read_excel(data_path / "median_house_price.xlsx", sheet_name="Monthly")
# Ensure the date column is datetime
if not pd.api.types.is_datetime64_any_dtype(prices["observation_date"]):
    prices["observation_date"] = pd.to_datetime(prices["observation_date"])

prices.set_axis(prices["observation_date"])  # index by date
# Read CPI file and parse dates; adjust column name if different
cpi_data = pd.read_excel(
    data_path / "cpi.xlsx",
    sheet_name="Monthly",
    parse_dates=["observation_date"]  # ensure this is the actual date column name
)

# Ensure the date column is datetime
if not pd.api.types.is_datetime64_any_dtype(cpi_data["observation_date"]):
    cpi_data["observation_date"] = pd.to_datetime(cpi_data["observation_date"])

# Filter CPI data to start from 1963-01-01 (inclusive)
cpi_data = cpi_data[cpi_data["observation_date"] >= "1963-01-01"]

# Extract CPI series (column named 'cpi' in the Excel file)
cpi = cpi_data["cpi"]

# Optional: reset index or keep dates as index
cpi = cpi.set_axis(cpi_data["observation_date"])  # index by date if desired

# Quick check
print(cpi.head())
print(f"Rows after filtering: {len(cpi)}")

# Calculate real house prices
cpi_100 = cpi / 100  # Convert CPI to index form
real_prices = prices.price.values / cpi_100.values  # Align CPI to prices index

# Read other datasets similarly and compute their monthly differences
loans = pd.read_excel(data_path / "loans.xlsx", sheet_name="Monthly")
loans = loans.set_index(loans["observation_date"])

net_wealth = pd.read_excel(data_path / "net_wealth_of_top_1.xlsx", sheet_name="Quarterly")
# trabsform to monthly with linear interpolation
net_wealth =net_wealth.set_index("observation_date")
# net_wealth rename percentage column to 'net_wealth
net_wealth.rename(columns={"percentage": "net_wealth"}, inplace=True)

net_wealth_monthly = net_wealth.resample('ME').interpolate(method='linear').ffill().bfill()

vacancy_rate = pd.read_excel(data_path / "vacancy_rate.xlsx", sheet_name="Quarterly")
vacancy_rate = vacancy_rate.set_index("observation_date")
vacancy_rate_monthly = vacancy_rate.resample('MS').interpolate(method='linear').ffill().bfill()



morgage_rate = pd.read_excel(data_path / "MORTGAGE30US.xlsx", sheet_name="Weekly, Ending Thursday")
morgage_rate = morgage_rate.set_index("observation_date")
# change column name to 'morgage_rate'
morgage_rate.rename(columns={"MORTGAGE30US": "morgage_rate"}, inplace=True)

# convert weekly to monthly
morgage_rate_monthly = morgage_rate.resample('MS').interpolate(method='linear').ffill().bfill()

## Import more data

pca_path = Path("data/PCA")
active_listings = pd.read_excel(pca_path / "ACTLISCOUUS.xlsx", sheet_name="Monthly")
active_listings.rename(columns={"ACTLISCOUUS": "active_listings"}, inplace=True)
active_listings.set_index("observation_date", inplace=True)


median_house_income = pd.read_excel(pca_path / "MEHOINUSA672N.xlsx", sheet_name="Annual")
median_house_income['observation_date'] = pd.to_datetime(median_house_income['observation_date'], errors='coerce')
median_house_income.set_index('observation_date', inplace=True)
median_house_income.rename(columns={"MEHOINUSA672N": "median_house_income"}, inplace=True)
median_house_income_monthly = median_house_income.resample('MS').interpolate(method='linear').ffill().bfill()

monthly_supply_homes = pd.read_excel(pca_path / "MSACSR.xlsx", sheet_name="Monthly")
monthly_supply_homes.set_index("observation_date", inplace=True)
monthly_supply_homes.rename(columns={"MSACSR": "monthly_supply_homes"}, inplace=True)

labor_share = pd.read_excel(pca_path / "PRS85006173.xlsx", sheet_name="Quarterly")
labor_share.set_index("observation_date", inplace=True)
labor_share.rename(columns={"PRS85006173": "labor_share"}, inplace=True)
labor_share_monthly = labor_share.resample('MS').interpolate(method='linear').ffill().bfill()

unemployment_rate = pd.read_excel(pca_path / "UNRATE.xlsx", sheet_name="Monthly")
unemployment_rate.set_index("observation_date", inplace=True)
unemployment_rate.rename(columns={"UNRATE": "unemployment_rate"}, inplace=True)

share_net_worth_bottom_50 = pd.read_excel(pca_path / "WFRBSB50215.xlsx", sheet_name="Quarterly")
share_net_worth_bottom_50.set_index("observation_date", inplace=True)
share_net_worth_bottom_50.rename(columns={"WFRBSB50215": "share_net_worth_bottom_50"}, inplace=True)
share_net_worth_bottom_50_monthly = share_net_worth_bottom_50.resample('MS').interpolate(method='linear').ffill().bfill()


# Calculate monthly differences
log_real_prices = np.log(real_prices)
log_real_prices = pd.Series(log_real_prices, index=prices.observation_date)

log_real_prices_diff = log_real_prices.diff().dropna()
loans_monthly_diff = loans.loans.diff().dropna()
net_wealth_monthly_diff = net_wealth_monthly.net_wealth.diff().dropna()
vacancy_rate_monthly_diff = vacancy_rate_monthly.vacancy_rate.diff().dropna()
morgage_rate_monthly_diff = morgage_rate_monthly.morgage_rate.diff().dropna()
active_listings_monthly_diff = active_listings.active_listings.diff().dropna()
median_house_income_monthly_diff = median_house_income_monthly.median_house_income.diff().dropna()
monthly_supply_homes_monthly_diff = monthly_supply_homes.monthly_supply_homes.diff().dropna()
labor_share_monthly_diff = labor_share_monthly.labor_share.diff().dropna()
unemployment_rate_monthly_diff = unemployment_rate.unemployment_rate.diff().dropna()
share_net_worth_bottom_50_monthly_diff = share_net_worth_bottom_50_monthly.share_net_worth_bottom_50.diff().dropna()    
# Combine all variables into a single DataFrame
# Align all series to the real_prices_diff datetime index and build the DataFrame
idx = prices.observation_date

data = pd.DataFrame({
    'real_prices_diff': log_real_prices_diff,
    'loans_diff': loans_monthly_diff.reindex(idx),
    'net_wealth_diff': net_wealth_monthly_diff.reindex(idx),
    'vacancy_rate_diff': vacancy_rate_monthly_diff.reindex(idx),
    'morgage_rate_diff': morgage_rate_monthly_diff.reindex(idx),
    'active_listings_diff': active_listings_monthly_diff.reindex(idx),
    'median_house_income_diff': median_house_income_monthly_diff.reindex(idx),
    'monthly_supply_homes_diff': monthly_supply_homes_monthly_diff.reindex(idx),
    'labor_share_diff': labor_share_monthly_diff.reindex(idx),
    'unemployment_rate_diff': unemployment_rate_monthly_diff.reindex(idx),
    'share_net_worth_bottom_50_diff': share_net_worth_bottom_50_monthly_diff.reindex(idx)
}, index=idx)







observation_date
1963-01-01    30.44
1963-02-01    30.48
1963-03-01    30.51
1963-04-01    30.48
1963-05-01    30.51
Name: cpi, dtype: float64
Rows after filtering: 752


AttributeError: 'NoneType' object has no attribute 'active_listings'

In [31]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np

X = data[[
'loans_diff',
'net_wealth_diff',
'vacancy_rate_diff',
'morgage_rate_diff',
'active_listings_diff',
'median_house_income_diff',
'monthly_supply_homes_diff',
'labor_share_diff',
'unemployment_rate_diff',
'share_net_worth_bottom_50_diff' ]]
y = data['real_prices_diff']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print("\n" + "=" * 60)
print("STEP 2: STANDARDIZATION")
print("=" * 60)
# Standardize features (required for PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeatures standardized (mean=0, std=1)")
print("Training set shape:", X_train_scaled.shape)
print("Test set shape:", X_test_scaled.shape)

print("\n" + "=" * 60)
print("STEP 3: PCA FACTOR ANALYSIS")
print("=" * 60)

# Perform PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Display explained variance
print("\nExplained variance by each component:")
for i, var in enumerate(pca.explained_variance_ratio_, 1):
    print(f"  PC{i}: {var:.4f} ({var*100:.2f}%)")

print(f"\nCumulative explained variance:")
cumsum = np.cumsum(pca.explained_variance_ratio_)
for i, var in enumerate(cumsum, 1):
    print(f"  PC1-PC{i}: {var:.4f} ({var*100:.2f}%)")

# Component loadings
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i}' for i in range(1, len(pca.components_) + 1)],
    index=X.columns
)
print("\nPCA Loadings (Component Matrix):")
print(loadings.round(3))

print("\n" + "=" * 60)
print("STEP 4: DETERMINE NUMBER OF FACTORS")
print("=" * 60)

# Kaiser criterion: eigenvalues > 1
eigenvalues = pca.explained_variance_
print("\nEigenvalues:")
for i, ev in enumerate(eigenvalues, 1):
    print(f"  PC{i}: {ev:.4f}", "✓ (Keep)" if ev > 1 else "  (Drop)")

n_components_kaiser = sum(eigenvalues > 1)
print(f"\nKaiser criterion suggests keeping {n_components_kaiser} component(s)")

# Choose based on variance threshold (e.g., 80%)
variance_threshold = 0.80
n_components_var = np.argmax(cumsum >= variance_threshold) + 1
print(f"To explain {variance_threshold*100}% variance: {n_components_var} component(s)")

print("\n" + "=" * 60)
print("STEP 5: REGRESSION WITH PRINCIPAL COMPONENTS")
print("=" * 60)

# Use the number of components that explain ~80-90% variance
n_final = max(n_components_kaiser, n_components_var)
print(f"\nUsing {n_final} principal component(s) for regression")

X_train_pca_reduced = X_train_pca[:, :n_final]
X_test_pca_reduced = X_test_pca[:, :n_final]

# Fit regression model
model = LinearRegression()
model.fit(X_train_pca_reduced, y_train)

# Predictions
y_train_pred = model.predict(X_train_pca_reduced)
y_test_pred = model.predict(X_test_pca_reduced)

# Evaluate
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\nModel Performance:")
print(f"  Training R²: {train_r2:.4f}")
print(f"  Test R²: {test_r2:.4f}")
print(f"  Training RMSE: {train_rmse:.6f}")
print(f"  Test RMSE: {test_rmse:.6f}")
print(f"  Training MAE: {train_mae:.6f}")
print(f"  Test MAE: {test_mae:.6f}")

print("\nRegression Coefficients (for PCs):")
for i, coef in enumerate(model.coef_, 1):
    print(f"  PC{i}: {coef:.4f}")
print(f"  Intercept: {model.intercept_:.6f}")

# Interpretation: Map PC coefficients back to original variables
print("\n" + "=" * 60)
print("INTERPRETATION: Effect on Price Changes")
print("=" * 60)
print("\nApproximate contribution of original variables:")
print("(PC coefficients × loadings)")

original_effects = np.dot(loadings.iloc[:, :n_final], model.coef_)
for var, effect in zip(X.columns, original_effects):
    print(f"  {var}: {effect:.4f}")

print("\n" + "=" * 60)
print("STEP 6: VISUALIZATION")
print("=" * 60)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Scree plot
axes[0, 0].bar(range(1, len(eigenvalues) + 1), eigenvalues, alpha=0.7, color='steelblue')
axes[0, 0].axhline(y=1, color='r', linestyle='--', label='Kaiser criterion', linewidth=2)
axes[0, 0].set_xlabel('Principal Component', fontsize=11)
axes[0, 0].set_ylabel('Eigenvalue', fontsize=11)
axes[0, 0].set_title('Scree Plot', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Cumulative variance explained
axes[0, 1].plot(range(1, len(cumsum) + 1), cumsum, marker='o', linewidth=2, 
                color='steelblue', markersize=8)
axes[0, 1].axhline(y=0.8, color='r', linestyle='--', label='80% variance', linewidth=2)
axes[0, 1].set_xlabel('Number of Components', fontsize=11)
axes[0, 1].set_ylabel('Cumulative Explained Variance', fontsize=11)
axes[0, 1].set_title('Cumulative Variance Explained', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim([0, 1.05])

# 3. Component loadings heatmap
sns.heatmap(loadings.iloc[:, :n_final], annot=True, cmap='RdBu_r', center=0, 
            ax=axes[1, 0], cbar_kws={'label': 'Loading'}, fmt='.3f',
            vmin=-1, vmax=1, linewidths=0.5)
axes[1, 0].set_title('PCA Loadings Heatmap', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Principal Component', fontsize=11)
axes[1, 0].set_ylabel('Original Variable (Differenced)', fontsize=11)

# 4. Actual vs Predicted
axes[1, 1].scatter(y_test, y_test_pred, alpha=0.6, color='steelblue', 
                   edgecolors='black', s=50)
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, 
                label='Perfect prediction')
axes[1, 1].set_xlabel('Actual Price Change (Log Diff)', fontsize=11)
axes[1, 1].set_ylabel('Predicted Price Change (Log Diff)', fontsize=11)
axes[1, 1].set_title(f'Actual vs Predicted (Test Set)\nR² = {test_r2:.4f}', 
                     fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pca_analysis_results.png', dpi=300, bbox_inches='tight')
print("\nVisualizations saved as 'pca_analysis_results.png'")
plt.show()

print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)
print("\nNote: You're modeling CHANGES in log prices using CHANGES in predictors.")
print("This is appropriate for non-stationary time series data.")

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.